In [ ]:
'''
Arquivos:
- Estabelecer a forma de guardar os arquivos conforme necessidade da CNN

Funções:
- Função de importação de arquivo com as seguintes colunas:
    Open High Low Close Volume + Indicadores
  Esta mesma função fará os índices das linhas serem as datas
- Função de escolha do intervalo de dias a terem o gráfico montado
- Função que gera o gráfico de candlestick com volume e indicadores

'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
from datetime import timedelta
from PIL import Image
from glob import glob


In [ ]:
'''
Importa os dados do ativo e renomeia as colunas
'''

asset = 'BTOW3'
time_frame = '1d'
file_name = asset + '_' + time_frame + '.xlsx'
path = '../collected_ data/' + file_name

inplace = True
df_input = pd.read_excel(path)
df_input.dropna(inplace = inplace)
df_input.set_index('Data', inplace = inplace)
df_input.sort_index(axis = 0, inplace = inplace)
df_input.rename(inplace = inplace, columns = {
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
    'Fechamento': 'Close',
    'Volume Financeiro': 'Volume',
    'Média Móvel E [9]': 'MME009',
    'Média Móvel A [21]': 'MMA021',
    'Média Móvel A [200]': 'MMA200',
    'Média Móvel A [51]': 'MMA051',
    'Média Móvel A [80]': 'MMA080',
    'Média Móvel A [400]': 'MMA400',
    'Média Móvel E [400]': 'MME400'})

# df_input.head()


In [ ]:
'''
Cria reordena as colunas dos indicadores em ordem alfabética
'''

df_candle = df_input[['Open', 'High', 'Low', 'Close', 'Volume']]
df_ind = df_input.reindex(sorted(df_input.columns[5:]), axis = 1)
df = pd.concat([df_candle, df_ind], axis=1)

# df.head()


In [ ]:
'''
Verifica se o dataframe reordenado é igual ao original
'''

columns = list(df.columns)
eq = True
for col in columns:
   eq = eq*df[col].equals(df_input[col])
if eq:
    print("Data Frame válido.")
else:
    print("Data Frame inválido!!!!")
    

In [ ]:
'''
Aprender a gerar os gráficos para então iterar os intervalos de data
'''

def date_delta(start_date, delta): # Função sem uso por enquanto
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days = delta)
    return end_date.strftime('%Y-%m-%d')


def clean_folder(path, match):
    files_list = glob(path + f'/*{match}*')
    # Escrever loop para deletar os arquivos da lista
    return


def gen_chart(df, date, trade_delta, min_profit, chart_delta, add, save_name):
    end_index = df.index.get_loc(date)
    start_index = end_index - chart_delta
    
    start_trade = df['Open'][end_index + 1]
    end_trade = df['Close'][end_index + trade_delta]
    if end_trade/start_trade >= 1 + min_profit/100:
        trade_class = 'long'
    elif end_trade/start_trade <= 1 - min_profit/100:
        trade_class = 'short'
    else:
        trade_class = 'wait'

    df_plot = df.iloc[start_index:end_index,:]
    erase_color = 'black'

    mc = mpf.make_marketcolors(up = 'g',down = 'r',
                               wick = 'inherit',
                               edge = 'inherit',
                               volume = 'grey')
    s = mpf.make_mpf_style(marketcolors = mc,
                           facecolor = erase_color,
                           edgecolor = erase_color,
                           figcolor = erase_color)
    addplot = mpf.make_addplot(df_plot[add])
    plot = mpf.plot(df_plot, type = 'candle',
                             volume = True,
                             style = s,
                             addplot = addplot,
                             savefig = '../generated_data/plot.png')

    uncropped_plot = Image.open('../generated_data/plot.png')
    box = (uncropped_plot.size[0]*0.202,
           uncropped_plot.size[1]*0.11,
           uncropped_plot.size[0]*0.88,
           uncropped_plot.size[1]*0.85)

    file_name = f'../generated_data/{save_name}_{trade_class}.png'
    cropped_plot = uncropped_plot.crop(box).save(file_name)
    return (start_trade, end_trade, trade_class)
    

In [ ]:
'''
Chamamento das funções de geração dos gráficos
Implementar o loop for
'''

date_range = pd.date_range(start='2020-02-01', end='2020-04-30')
trade_delta = 3
min_profit = 1 # in %
chart_delta = 22
ind_columns = list(df.columns)[5:9]

for date in date_range:
    try:
        file_date = date.strftime('%Y-%m-%d')
        save_name = f'{asset}_{file_date}'
        out = gen_chart(df, date, trade_delta, min_profit, chart_delta, ind_columns, save_name)
        print(f'{file_date} {out[0]} {out[1]} {out[2]}')
    except KeyError:
        continue
    except IndexError:
        continue


In [ ]:
'''
Escrever função que organiza aleatoriamente os arquivos em pastas
Pastas: train, valid e test

Cada classe terá o mesmo número de exemplos. Avaliar possibilidade de copiar ao invés de mover.
Dica: files_distribution = [[long, n_long]],
                            [short, n_short],
                            [wait, n_wait]]
      for class in files_distribution:
          for file in random.sample(glob.glob(class[0] + '.*'), class[1]):
              shutil.move(file, '/charts/' + class[0])
'''